In [107]:
import pandas as pd 
import numpy as np
from datetime import datetime
import joblib
import os
import pickle

In [108]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
print(ticker.options)

('2025-08-08', '2025-08-15', '2025-08-22', '2025-08-29', '2025-09-05', '2025-09-19', '2025-10-17', '2025-11-21', '2025-12-19', '2026-01-16', '2026-02-20', '2026-03-20', '2026-05-15', '2026-06-18', '2026-09-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')


In [109]:
# Create and load the ticker
ticker_symbol = "AAPL"
expiration_index = 0
ticker = yf.Ticker(ticker_symbol)
expirations = ticker.options
selected_expiration = '2026-02-20'

# Pull price history
hist = ticker.history(period='1mo')
current_price = hist['Close'].iloc[-1]

# Estimate volatility (anualized)
log_returns = np.log(hist['Close'] / hist['Close'].shift(1)).dropna()
volatility = log_returns.std() * np.sqrt(252)

# Risk-free rate (static from API)
risk_free_rate = 0.045

#strike
strike_range = np.round(np.linspace(current_price * 0.8, current_price * 1.2, 10), 2)
#volatility
vol_range = np.round(np.linspace(volatility * 0.5, max(0.6, volatility * 2.0), 10), 2)


# Time to maturity
expiry_date = pd.to_datetime(selected_expiration)
today = pd.to_datetime("today").normalize()  # remove timestamp
ttm = (expiry_date - today).days / 365
print("Time to Maturity (ttm):", ttm)


#output df
summary = pd.DataFrame([{
    'Ticker:': ticker_symbol,
    'Current Price': round(current_price, 2),
    'Estimated Volatility (Annualized)': round(volatility, 4),
    'Risk-Free Rate:': risk_free_rate,
    'Strike Range Example:': strike_range.tolist(),
    'Volatility Range Example:': vol_range.tolist()
}])

display(summary)

# Save params for later use
os.makedirs("data", exist_ok=True)
params = {
    'ticker_symbol': ticker_symbol,
    'selected_expiration': selected_expiration,
    'current_price': current_price,
    'volatility': volatility,
    'risk_free_rate': risk_free_rate,
    'strike_range': strike_range,
    'vol_range': vol_range,
    'ttm': ttm
}
with open("../data/params.pkl", "wb") as f:
    pickle.dump(params, f)
print("Saved all parameters to ../data/params.pkl")

Time to Maturity (ttm): 0.5534246575342465


,Ticker:,Current Price,Estimated Volatility (Annualized),Risk-Free Rate:,Strike Range Example:,Volatility Range Example:
0,AAPL,202.38,0.1424,0.045,"[161.9, 170.9, 179.89, 188.89, 197.88, 206.88,...","[0.07, 0.13, 0.19, 0.25, 0.31, 0.36, 0.42, 0.4..."


Saved all parameters to ../data/params.pkl
